# ⚯ Pair Programming 3.1 - Hypothesis Testing

In this exercise, you will have the opportunity to decide to pick the variables that you want to perform one and two sample t-tests.

## Objectives

You will be able to:

* Construct and interpret a confidence interval for a single mean
* Perform a hypothesis test for a single mean
* Construct and interpret a confidence interval for two independent means
* Perform a hypothesis test for two independent means


## The Data: Last Words of Death Row Inmates

On Kaggle there is a famous data set of *[Last Words of Death Row Inmates](https://www.kaggle.com/datasets/mykhe1097/last-words-of-death-row-inmates)*. While this data set is often used for natural language processing, we'll use the non-text data for statistical inference on one and two means using the t-distribution.

### Data Dictionary

The dataset consists of 545 observations with 21 variables. They are:

* `Execution`: The order of execution, numeric.
* `LastName`: Last name of the offender, character.
* `FirstName`: First name of the offender, character.
* `TDCJNumber`: TDCJ Number of the offender, numeric.
* `Age`: Age of the offender, numeric.
* `Race`: Race of the offender, categorical : Black, Hispanic, White, Other.
* `CountyOfConviction`: County of conviction, character.
* `AgeWhenReceived`: Age of offender when received, numeric.
* `EducationLevel`: Education level of offender, numeric.
* `NativeCounty`: Native county of offender, categorical : 0 = Within Texas, 1= Outside Texas.
* `PreviousCrime`: Whether the offender committed any crime before, categorical: 0= No, 1= Yes.
* `Codefendants`: Number of co-defendants, numeric.
* `NumberVictim`: Number of victims, numeric.
* `WhiteVictim`, `HispanicVictim`, `BlackVictim`, `VictimOtherRace` `FemaleVictim`, `MaleVictim`: Number of victims with specified demographic features, numeric.
* `LastStatement`: Last statement of offender, character.

The [data file](https://github.com/learn-co-curriculum/dsc-hypothesis-testing-ent/blob/main/data/texas.csv) that we'll be using has the first 20 variables, i.e. we're excluding the `LastStatement` variable.

*N.B.*: Note this data is from 1982 - 2017 and for that time period the data is exhaustive.

## Preliminary Steps

As usual, begin with importing the Python libraries that you will need. Then bring in the data and explore it.

You'll likely want to use `scipy.stats`. If you want to use `confidence_interval(confidence_level=0.95)` with [`scipy.stats.ttest_ind`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html), then you need to have `scipy` version 1.11.0.

In [52]:
# Pip install version 1.11.0 of scipy (if necessary)
# Code here

! pip install scipy==1.10.0

  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\nisalas\\AppData\\Local\\anaconda3\\envs\\learn-env\\Lib\\site-packages\\~umpy\\.libs\\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



In [1]:
# Bring in germane libraries; you may need to require scipy 11.1.0
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stats
scipy.__version__
# Code here

'1.10.0'

In [3]:

scipy.__version__

'1.10.0'

In [4]:
# Your code here to bring in the data

df = pd.read_csv('data/texas.csv')

Now that you have brought in the data. Consider these or similar commands to get a feel for the data.
* `.info()`
* `.head()`



In [5]:
# Your code here; you may want to use multiple code chunks for .info() and .head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383 entries, 0 to 382
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   LastName            383 non-null    object
 1   FirstName           383 non-null    object
 2   TDCJNumber          383 non-null    int64 
 3   Age                 383 non-null    int64 
 4   Race                383 non-null    object
 5   CountyOfConviction  383 non-null    object
 6   AgeWhenReceived     383 non-null    int64 
 7   EducationLevel      383 non-null    int64 
 8   NativeCounty        383 non-null    int64 
 9   PreviousCrime       383 non-null    int64 
 10  Codefendants        383 non-null    int64 
 11  NumberVictim        383 non-null    int64 
 12  WhiteVictim         383 non-null    int64 
 13  HispanicVictim      383 non-null    int64 
 14  BlackVictim         383 non-null    int64 
 15  VictimOther.Races   383 non-null    int64 
 16  FemaleVictim        383 no

In [6]:
df.head()

,LastName,FirstName,TDCJNumber,Age,Race,CountyOfConviction,AgeWhenReceived,EducationLevel,NativeCounty,PreviousCrime,Codefendants,NumberVictim,WhiteVictim,HispanicVictim,BlackVictim,VictimOther.Races,FemaleVictim,MaleVictim
0,Cardenas,Ruben,999275,47,Hispanic,Hidalgo,28,11,1,0,0,1,0,1,0,0,1,0
1,Pruett,Robert,999411,38,White,Bee,22,8,0,1,0,1,1,0,0,0,0,1
2,Preyor,Taichin,999494,46,Black,Bexar,34,10,0,0,0,1,1,0,0,0,0,1
3,Bigby,James,997,61,White,Tarrant,36,9,0,1,0,1,1,0,0,0,0,1
4,Ruiz,Rolando,999145,44,Hispanic,Bexar,22,10,0,1,4,1,0,1,0,0,1,0


### Preprocessing

While there are a number of preprocessing steps you may choose to make, at the very least you should do the following.

Based on the data dictionary and how the data is interpreted by Numpy, convert any non-numeric data into the proper form. Verify that your code worked correctly.

In [24]:
# Your code here; you may want to use multiple code chunks
df['PreviousCrime'] = df['PreviousCrime'].replace({0: 'No', 1: 'Yes'})
df['NativeCounty'] = df['NativeCounty'].replace({0: 'No', 1: 'Yes'})

In [26]:
df.head()

,LastName,FirstName,TDCJNumber,Age,Race,CountyOfConviction,AgeWhenReceived,EducationLevel,NativeCounty,PreviousCrime,Codefendants,NumberVictim,WhiteVictim,HispanicVictim,BlackVictim,VictimOther.Races,FemaleVictim,MaleVictim
0,Cardenas,Ruben,999275,47,Hispanic,Hidalgo,28,11,Yes,No,0,1,0,1,0,0,1,0
1,Pruett,Robert,999411,38,White,Bee,22,8,No,Yes,0,1,1,0,0,0,0,1
2,Preyor,Taichin,999494,46,Black,Bexar,34,10,No,No,0,1,1,0,0,0,0,1
3,Bigby,James,997,61,White,Tarrant,36,9,No,Yes,0,1,1,0,0,0,0,1
4,Ruiz,Rolando,999145,44,Hispanic,Bexar,22,10,No,Yes,4,1,0,1,0,0,1,0


### EDA

Now perform Exploratory Data Analysis of at least three variables of your choosing. You should look at both descriptive statistics and visualizations.

In [7]:
# Your code here; you may want to use multiple code chunks

df['AgeWhenReceived'].mean()

28.503916449086162

In [8]:
df['CountyOfConviction'].value_counts()

Harris          73
Dallas          34
Tarrant         32
Bexar           30
Montgomery      12
                ..
Val Verde        1
Dawson           1
Freestone        1
Kendall          1
San Patricio     1
Name: CountyOfConviction, Length: 82, dtype: int64

In [9]:
df['CountyOfConviction'].value_counts()[:10]

Harris        73
Dallas        34
Tarrant       32
Bexar         30
Montgomery    12
Nueces        11
Jefferson     11
Smith         11
Brazos        11
Lubbock        8
Name: CountyOfConviction, dtype: int64

<BarContainer object of 6 artists>

In [10]:
plt.bar(df['CountyOfConviction'].value_counts()[:6].index, df['CountyOfConviction'].value_counts()[:6])
 

## Statistical Inference

### One mean

Now pick a variable that you can perform statistical inference on a single sample mean using the t-distribution.

In the code block below please respond to all of the following (commented out):

*   What is the variable you chose?
*   How do you know it is appropriate for statistical inference on a single mean using the t-distribution?
*   You'll be asked to contruct and interpret a confidence interval. What is the level of confidence that you'll pick?
*   You'll be asked to perform a hypothesis test.
  *   What are the null and alternative hypotheses?
  *   What is the level of significance?



In [19]:
# Type your responses to the bullet points here
# Age
# We can take a sample less than 30 
# Level of confidence .95
# Null: There is no difference in the ages sample and population
# Alt: There is a difference is the ages sample and population 

In [20]:
#AgeWhenReceived
#we have the full dataset
#level of confidence .95
# Null: There is no difference in theAgeWhenReceived sample and population
# Alt: There is a difference is the AgeWhenReceived sample and population 

#### Confidence Interval

Now construct a confidence interval for your chosen variable at the level of confidence you chose above.

In [21]:
# Type your code for the confidence interval here.

Age_CI = stats.ttest_1samp(df.Age, popmean=df['Age'].mean())
Age_CI.confidence_interval(confidence_level=0.95)

ConfidenceInterval(low=38.503965849221714, high=40.24277044320648)

In [25]:
Age_when_con_CI = stats.ttest_1samp(df.AgeWhenReceived, popmean=df['AgeWhenReceived'].mean())
Age_when_con_CI.confidence_interval(confidence_level=0.95)

ConfidenceInterval(low=27.68190973621669, high=29.325923161955636)

Interpret the confidence interval in the code block below (commented out).

In [ ]:
# Interpret the confidence interval (this is true for both Age and AgeWhenReceived)
# if were to pull samples it is expected that 95% to be within the pop mean 

#### Hypothesis Test

Now perform a hypothesis test for your chosen variable at the level of significance you chose above.

In [ ]:
print(df['Age'].mean())
age = stats.ttest_1samp(df['Age'], popmean = 39)

age

In [28]:
df['AgeWhenReceived'].mean()

28.503916449086162

In [30]:
# Type your code for the hypothesis test here.

print(df['AgeWhenReceived'].mean())
age = stats.ttest_1samp(df['AgeWhenReceived'], popmean = 28)

age

28.503916449086162


TtestResult(statistic=1.2053396093540385, pvalue=0.22881834497084155, df=382)

Draw the appropriate conclusion for the hypothesis test and justify your answer in the code block below (commented out).

In [13]:
# Draw the conclusion from the hypothesis test (this is the same for Age and AgeWhenRecieved )
# we fail to reject the null hypothesus because the pvalue was greater 

### Two means

Now pick a variable that you can perform statistical inference on two (independent) sample means using the t-distribution.

In the code block below please respond to all of the following (commented out):

*   What are the variables you chose?
*   How do you know it is appropriate for statistical inference on two (independent)means using the t-distribution?
*   You'll be asked to contruct and interpret a confidence interval. What is the level of confidence that you'll pick?
*   You'll be asked to perform a hypothesis test.
  *   What are the null and alternative hypotheses?
  *   What is the level of significance?

In [ ]:
# Type your responses to the bullet points here

In [17]:
pc = df[df['PreviousCrime'] == 'Yes']
npc = df[df['PreviousCrime'] == 'No']

#### Confidence Interval

Now construct a confidence interval for your chosen variables at the level of confidence you chose above.

In [ ]:
# Type your code for the confidence interval here.

Interpret the confidence interval in the code block below (commented out).

In [ ]:
# Interpret the confidence interval

#### Hypothesis Test

Now perform a hypothesis test for your chosen variable at the level of significance you chose above.

In [ ]:
# Type your code for the hypothesis test here.

Draw the appropriate conclusion for the hypothesis test and justify your answer in the code block below (commented out).

In [ ]:
# Draw the conclusion from the hypothesis test

## Summary

Using a multivariate data set, you were able to pick variables to perform one and two sample t confidence intervals and t-tests.